## The Research Example in Chapter 8, Section 6: Automatic Offsets

In [17]:
import numpy as np
import astropy.units as u
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord

class NoMatchesError(Exception):
    pass

Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"
Gaia.ROW_LIMIT = 500

def find_offset_star(target_coordinate:SkyCoord,
                     cone_radius:u.Quantity = 2*u.arcmin,
                     limiting_magnitude:float=10.0):
    j = Gaia.cone_search_async(target_coordinate,
                               radius=cone_radius)
    r = j.get_results()
    r = r[r['phot_g_mean_mag'] < limiting_magnitude]
    if len(r) == 0:
        raise NoMatchesError('No stars match the search criteria')
    star_coords = SkyCoord(ra=r['ra'], dec=r['dec'], unit='deg')
    all_separations = target_coordinate.separation(star_coords)
    ind = np.argmin(all_separations)
    best_star_coord = SkyCoord(r['ra'][ind], dec=r['dec'][ind], unit='deg')
    final_offset = best_star_coord.spherical_offsets_to(target_coordinate)
    offsets_arcsec = [i.to(u.arcsec) for i in final_offset]
    return best_star_coord, offsets_arcsec

coordinate = SkyCoord.from_name('M81')
coord_star, os = find_offset_star(coordinate)


INFO: Query finished. [astroquery.utils.tap.core]


NoMatchesError: No stars match the search criteria